<h1>Problem Description: </h1>

Classification of Web page content is vital to many tasks in Web information retrieval such as maintaining Web directories and focused crawling. The uncontrolled nature of Web content presents additional challenges to Web page classification as compared to traditional text classification, however the interconnected nature of hypertext also provides features that can assist the process.
Here the task is to classify the web pages to the respective classes it belongs to, in a single label classification setup (Each webpage can belong to only 1 class).

Basically given the complete html and url, predict the tag a web page belongs to out of 9 predefined tags as given below:

People profile 2) Conferences/Congress 3) Forums 4) News article 5) Clinical trials 6) Publication 7) Thesis 8) Guidelines 9) Others

In [0]:
#Connecting Google Colab with Google Drive

!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}


Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [0]:
!mkdir -p drive
!google-drive-ocamlfuse drive

In [0]:
!ls  /content/drive/Hackthon/Innoplexus/temp/

glove.6B.100d.txt  new_data.csv  test_nvPHrOx.csv  train.csv


In [0]:
#unzip Autopilot folder as data is inside there
!cp  /content/drive/Hackthon/Innoplexus/temp/* .
#!cp new_html.csv  /content/drive/Hackthon/Innoplexus/

In [0]:
#!unzip train.zip
#!unzip glove.6B.zip

Archive:  train.zip
  inflating: train.csv               
  inflating: html_data.csv           
Archive:  glove.6B.zip
  inflating: glove.6B.50d.txt        
  inflating: glove.6B.100d.txt       
  inflating: glove.6B.200d.txt       
  inflating: glove.6B.300d.txt       


In [0]:
!ls

datalab  drive	glove.6B.100d.txt  new_data.csv  test_nvPHrOx.csv  train.csv


In [0]:
#!cp glove.6B.100d.txt  /content/drive/Hackthon/Innoplexus/temp/

In [0]:
#Import All Required Python Modules

import pandas as pd
import numpy as np
import re
from keras.models import Sequential
from keras.layers import Dropout
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from keras.layers import Dense, Input,  Flatten
from keras.layers import Conv1D, MaxPooling1D, AveragePooling1D, Embedding

from keras.callbacks import ModelCheckpoint
from sklearn.metrics import confusion_matrix

from numpy.random import RandomState
from sklearn.preprocessing import LabelEncoder
from keras.layers import LSTM,GRU
SEED = RandomState(1234567890)

Using TensorFlow backend.


In [0]:
#Get Train and Test data from csv file

with open('new_data.csv', encoding='ascii', errors='ignore') as infile:
    Data = pd.read_csv(infile)
Data_Train = pd.read_csv("train.csv")
Data_Test  = pd.read_csv("test_nvPHrOx.csv")

In [0]:
Data.shape

(79345, 2)

In [0]:
# Data Cleaning 

def url_str_conv(s):
  #ss =re.sub('http://\w.*/','',s)
  #ss =re.sub('https://\w.*/','',s)
  i1 = s.find("//")
  i2 = s.find("/",i1+2)
  if i1==-1 or i2 == -1:
    i3 = s.find(".com")
    ss = s[i1+2:i3]
    ss =re.sub('www.','',ss)
    #print(ss)
    ss =re.findall(r'\w\w+',ss)
    ss = ' '.join(ss)
    return ss
  ss = s[i2+1:]
  ss =re.sub('www.','',ss)
  ss =re.findall(r'\w\w+',ss)
  ss = ' '.join(ss)
  return ss

In [0]:
def title_text(s):
  s = str(s)
  ss =re.sub('\d',' ',s)
  ss =re.findall(r'\w\w+',ss)
  ss = ' '.join(ss)
  ss = ss.strip()
  return ss

In [0]:
q= 'https://www.clinicaltrialsregister.eu/ctr-search/trial/2006-006214-16/GB'
title_text(q)

'https www clinicaltrialsregister eu ctr search trial GB'

In [0]:
Data['new_title'] = list(map(title_text,Data.Title))
Data_Train['new_url'] = list(map(url_str_conv,Data_Train.Url))
Data_Test['new_url'] = list(map(url_str_conv,Data_Test.Url))
#

In [0]:
Data_Train = pd.merge(Data_Train,Data,how="left",on="Webpage_id",)
Data_Test = pd.merge(Data_Test,Data,how="left",on="Webpage_id")
#Data_Train.drop(columns=["Title_y","new_Title_y"])

In [0]:
def fill_empty(new_title,new_url):
    if len(new_title) < 2:
      return new_url
    else:
      return new_title
      

In [0]:
Data_Train["update_title"] = list(map(fill_empty,Data_Train.new_title,Data_Train.new_url))
Data_Test["update_title"] = list(map(fill_empty,Data_Test.new_title,Data_Test.new_url))

In [0]:
Data_Train.head()

,Webpage_id,Domain,Url,Tag,new_url,Title,new_title,update_title
0,1,www.fiercepharma.com,http://www.fiercepharma.com/marketing/tecfider...,news,marketing tecfidera gilenya and aubagio way ba...,"Tecfidera, Gilenya and Aubagio&#039;s 3-way ba...",Tecfidera Gilenya and Aubagio way battle for M...,Tecfidera Gilenya and Aubagio way battle for M...
1,2,www.fiercepharma.com,http://www.fiercepharma.com/pharma/novo-equipp...,news,pharma novo equipped to weather storm diabetes...,novo equipped to weather the storm in the u.s....,novo equipped to weather the storm in the diab...,novo equipped to weather the storm in the diab...
2,3,www.fiercepharma.com,http://www.fiercepharma.com/pharma/another-exe...,news,pharma another exec departs troubled endo and ...,another exec departs troubled endo--and this t...,another exec departs troubled endo and this ti...,another exec departs troubled endo and this ti...
3,4,www.fiercepharma.com,http://www.fiercepharma.com/pharma/teva-buy-bi...,news,pharma teva buy biosim specialist celltrion it...,would teva buy korea&#039;s celltrion to beef ...,would teva buy korea celltrion to beef up in b...,would teva buy korea celltrion to beef up in b...
4,5,www.fiercepharma.com,http://www.fiercepharma.com/marketing/actress-...,news,marketing actress marissa tomei partners aller...,restasis-maker allergan recruits actress maris...,restasis maker allergan recruits actress maris...,restasis maker allergan recruits actress maris...


In [0]:
Data_Train.sort_values('Tag',inplace=True)

In [0]:
from sklearn.preprocessing import LabelEncoder

In [0]:
#Method to do Tokenization for Train DataSet

def tokenize_data(X_raw, Y_raw):
    tokenizer = Tokenizer(num_words=MAX_NB_WORDS,char_level=True)
    tokenizer.fit_on_texts(X_raw)
    sequences = tokenizer.texts_to_sequences(X_raw)
    word_index = tokenizer.word_index
    X_processed = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
    label_encoder_y = LabelEncoder()
    Y_processed=to_categorical(label_encoder_y.fit_transform(Y_raw))

    return X_processed, Y_processed, word_index,tokenizer,label_encoder_y

In [0]:
#Method to do Tokenization for Test DataSet

def tokenize_test_data(X_raw, tokenizer):
    sequences = tokenizer.texts_to_sequences(X_raw)
    word_index = tokenizer.word_index
    X_processed = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)

    return X_processed,word_index

In [0]:
#Spliting Train and Dev dataset

def split_the_data(X_processed, Y_processed):
  count=1
  #X=list(X_processed)
  #Y=list(Y_processed)
  x_train=[]
  y_train=[]
  x_val=[]
  y_val=[]
  for i,j in zip(X_processed,Y_processed):
    if count%3==0:
      x_val.append(i)
      y_val.append(j)
    else:
      x_train.append(i)
      y_train.append(j)
    count+=1
  x_train=np.asarray(x_train)
  x_val=np.asarray(x_val)
  y_train=np.asarray(y_train)
  y_val=np.asarray(y_val)
  return x_train, x_val, y_train, y_val

In [0]:
#Word Embedding with Glove Word Embeddings

def make_embedding_layer(word_index):
    embeddings = get_embeddings()
    nb_words = min(MAX_NB_WORDS, len(word_index))
    embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))

    for word, i in word_index.items():
        if i >= MAX_NB_WORDS:
            continue
        embedding_vector = embeddings.get(word)
        if embedding_vector is not None:
            embedding_matrix[i] = embedding_vector

    embedding_layer = Embedding(nb_words, EMBEDDING_DIM, weights=[embedding_matrix], input_length=MAX_SEQUENCE_LENGTH, trainable=False)
    return embedding_layer

In [0]:
def get_embeddings():
    embeddings = {}
    with open(GLOVE_FILE, 'r') as f:
        for line in f:
            values = line.split()
            word = values[0]
            coefs = np.asarray(values[1:], dtype='float32')
            embeddings[word] = coefs
    return embeddings

In [0]:
MAX_SEQUENCE_LENGTH = 500
MAX_NB_WORDS = 20000
EMBEDDING_DIM = 100
GLOVE_FILE = 'glove.6B.100d.txt'

In [0]:
X_processed, Y_processed, word_index,tokenizer,label_encoder_y = tokenize_data(Data_Train.update_title,Data_Train.Tag)
x_train, x_val, y_train, y_val = split_the_data(X_processed, Y_processed)
#embedded_sequences = make_embedding_layer(word_index)
nb_words = min(MAX_NB_WORDS, len(word_index))

In [0]:
def make_model_lstm(labels_index,drop_rate=0.5,unit=100,dense=256):
    
    model = Sequential([
        Embedding(nb_words, EMBEDDING_DIM, input_length=MAX_SEQUENCE_LENGTH),
        LSTM(unit),
        #LSTM(unit,return_sequences=True),
        #LSTM(unit),
        #Flatten(),
        #Dropout(drop_rate),
        #Dense(dense, activation='relu'),
        #Dropout(drop_rate),
        #Dense(500, activation='relu'),
        #Dropout(drop_rate),
        #Dense(100, activation='relu'),
        #Dropout(drop_rate),
        #Dense(28, activation='relu'),
        Dense(labels_index, activation='softmax')
    ])

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

    return model

In [0]:
def make_model_gru(labels_index,drop_rate=0.5,unit=100,dense=256):
    
    model = Sequential([
        embedded_sequences,
        GRU(unit),
        #GRU(50),
        #Flatten(),
        Dropout(drop_rate),
        Dense(dense, activation='relu'),
        Dropout(drop_rate),
        Dense(64, activation='relu'),
        Dense(labels_index, activation='softmax')
    ])

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['acc'])

    return model

In [0]:
labels_index=9
model = make_model_lstm(labels_index,drop_rate=0.3,unit=50,dense=1000)
model.fit(x_train, y_train, validation_data=(x_val, y_val),epochs=30, batch_size=256)

Train on 35632 samples, validate on 17815 samples
Epoch 1/30
35632/35632 [==============================] - 193s 5ms/step - loss: 1.8411 - acc: 0.3643 - val_loss: 1.5913 - val_acc: 0.4297
Epoch 2/30
35632/35632 [==============================] - 192s 5ms/step - loss: 1.4882 - acc: 0.4659 - val_loss: 1.3993 - val_acc: 0.5017
Epoch 3/30
35632/35632 [==============================] - 194s 5ms/step - loss: 1.3363 - acc: 0.5269 - val_loss: 1.2852 - val_acc: 0.5415
Epoch 4/30
35632/35632 [==============================] - 193s 5ms/step - loss: 1.2257 - acc: 0.5634 - val_loss: 1.1760 - val_acc: 0.5853
Epoch 5/30
35632/35632 [==============================] - 193s 5ms/step - loss: 1.1339 - acc: 0.5984 - val_loss: 1.1839 - val_acc: 0.5784
Epoch 6/30
35632/35632 [==============================] - 193s 5ms/step - loss: 1.0707 - acc: 0.6309 - val_loss: 1.0343 - val_acc: 0.6446
Epoch 7/30
35632/35632 [==============================] - 193s 5ms/step - loss: 1.0017 - acc: 0.6585 - val_loss: 0.9911 - 

In [0]:
"""

l
abels_index=9
model = make_model_lstm(labels_index, embedded_sequences,drop_rate=0.8,unit=500,dense=100)
model.fit(x_train, y_train, validation_data=(x_val, y_val),epochs=15, batch_size=256,)
model = make_model_lstm(labels_index, embedded_sequences,drop_rate=0.7,unit=500,dense=500)
model.fit(x_train, y_train, validation_data=(x_val, y_val),epochs=15, batch_size=256,)
model = make_model_lstm(labels_index, embedded_sequences,drop_rate=0.6,unit=500,dense=500)
model.fit(x_train, y_train, validation_data=(x_val, y_val),epochs=15, batch_size=256,)
model = make_model_lstm(labels_index, embedded_sequences,drop_rate=0.3,unit=50,dense=100)
model.fit(x_train, y_train, validation_data=(x_val, y_val),epochs=15, batch_size=256,)
"""

Train on 35632 samples, validate on 17815 samples
Epoch 1/15
35632/35632 [==============================] - 48s 1ms/step - loss: 1.4153 - acc: 0.4990 - val_loss: 0.9163 - val_acc: 0.6955
Epoch 2/15
35632/35632 [==============================] - 46s 1ms/step - loss: 0.9440 - acc: 0.6740 - val_loss: 0.7370 - val_acc: 0.7491
Epoch 3/15
35632/35632 [==============================] - 46s 1ms/step - loss: 0.7898 - acc: 0.7320 - val_loss: 0.6355 - val_acc: 0.7884
Epoch 4/15
35632/35632 [==============================] - 46s 1ms/step - loss: 0.7058 - acc: 0.7633 - val_loss: 0.5988 - val_acc: 0.8006
Epoch 5/15
35632/35632 [==============================] - 46s 1ms/step - loss: 0.6372 - acc: 0.7863 - val_loss: 0.5534 - val_acc: 0.8191
Epoch 6/15
35632/35632 [==============================] - 46s 1ms/step - loss: 0.5831 - acc: 0.8059 - val_loss: 0.5097 - val_acc: 0.8315
Epoch 7/15
35632/35632 [==============================] - 46s 1ms/step - loss: 0.5478 - acc: 0.8205 - val_loss: 0.5044 - val_acc

In [0]:
"""
model = make_model_gru(labels_index, embedded_sequences,drop_rate=0.8,unit=500,dense=500)
model.fit(x_train, y_train, validation_data=(x_val, y_val),epochs=15, batch_size=256,)
model = make_model_gru(labels_index, embedded_sequences,drop_rate=0.7,unit=500,dense=500)
model.fit(x_train, y_train, validation_data=(x_val, y_val),epochs=15, batch_size=256,)
model = make_model_gru(labels_index, embedded_sequences,drop_rate=0.6,unit=500,dense=500)
model.fit(x_train, y_train, validation_data=(x_val, y_val),epochs=15, batch_size=256,)
model = make_model_gru(labels_index, embedded_sequences,drop_rate=0.3,unit=50,dense=100)
model.fit(x_train, y_train, validation_data=(x_val, y_val),epochs=15, batch_size=256,)
"""

Train on 35632 samples, validate on 17815 samples
Epoch 1/15
35632/35632 [==============================] - 38s 1ms/step - loss: 1.4521 - acc: 0.4805 - val_loss: 0.8922 - val_acc: 0.6980
Epoch 2/15
35632/35632 [==============================] - 36s 1ms/step - loss: 0.9273 - acc: 0.6775 - val_loss: 0.6930 - val_acc: 0.7653
Epoch 3/15
35632/35632 [==============================] - 36s 1ms/step - loss: 0.7524 - acc: 0.7463 - val_loss: 0.6137 - val_acc: 0.7938
Epoch 4/15
35632/35632 [==============================] - 36s 1ms/step - loss: 1.1539 - acc: 0.6551 - val_loss: 0.6321 - val_acc: 0.7967
Epoch 5/15
35632/35632 [==============================] - 36s 1ms/step - loss: 0.8613 - acc: 0.7252 - val_loss: 0.6307 - val_acc: 0.8009
Epoch 6/15
35632/35632 [==============================] - 36s 1ms/step - loss: 0.6544 - acc: 0.7868 - val_loss: 0.5323 - val_acc: 0.8314
Epoch 7/15
35632/35632 [==============================] - 36s 1ms/step - loss: 0.5671 - acc: 0.8161 - val_loss: 0.5007 - val_acc

In [0]:
#labels_index=9
#model = make_model_lstm(labels_index, embedded_sequences)
#model.fit(x_train, y_train, validation_data=(x_val, y_val),epochs=15, batch_size=256,)

In [0]:
model.save("gru_100_ep_30_model.h5")
!cp gru_100_ep_30_title_model.h5  /content/drive/Hackthon/Innoplexus/

In [0]:
from sklearn.metrics import confusion_matrix, f1_score, precision_score, recall_score

In [0]:
s = [np.argmax(y, axis=None, out=None) for y in y_val]

In [0]:
#pre_train = model.predict_classes(x_train)
pre_val = model.predict_classes(x_val)

In [0]:
print(f1_score(s,pre_val,average='weighted'))
print(confusion_matrix(s,pre_val,))

0.9261524421751554

In [0]:
Data_test = Data_Test

In [0]:
#Data_test = pd.read_csv("test_nvPHrOx.csv")

In [0]:
Data_test.shape

(25787, 7)

In [0]:
Data_test.head()

,Webpage_id,Domain,Url,new_url,Title,new_title,update_title
0,31,isrctn.com,http://www.isrctn.com/ISRCTN57801413,ISRCTN57801413,isrctn - isrctn57801413: development and valid...,isrctn isrctn development and validation of ca...,isrctn isrctn development and validation of ca...
1,32,www.clinicaltrialsregister.eu,https://www.clinicaltrialsregister.eu/ctr-sear...,ctr search trial 2006 006214 16 GB,clinical trials register,clinical trials register,clinical trials register
2,33,www.clinicaltrialsregister.eu,https://www.clinicaltrialsregister.eu/ctr-sear...,ctr search trial 2006 004265 34 LT,clinical trials register,clinical trials register,clinical trials register
3,34,www.clinicaltrialsregister.eu,https://www.clinicaltrialsregister.eu/ctr-sear...,ctr search trial 2010 022183 12 IT,clinical trials register,clinical trials register,clinical trials register
4,35,www.clinicaltrialsregister.eu,https://www.clinicaltrialsregister.eu/ctr-sear...,ctr search trial 2010 021349 36 NL,clinical trials register,clinical trials register,clinical trials register


In [0]:

#Data_test['new_url'] = list(map(url_str_conv,Data_test.Url))


In [0]:
x_test,word_index = tokenize_test_data(Data_test['update_title'], tokenizer)

In [0]:
type(x_test)

numpy.ndarray

In [0]:
y_test = model.predict_classes(x_test)

In [0]:
s =label_encoder_y.inverse_transform(list(y_test))

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [0]:
s

array(['others', 'clinicalTrials', 'clinicalTrials', ..., 'profile',
       'others', 'profile'], dtype=object)

In [0]:
Data_test["Tag"] = s

In [0]:
Data_test.to_csv("test_predicted_GRU.csv",index=False,columns=['Webpage_id',"Tag"])

In [0]:
from google.colab import files
files.download("test_predicted_GRU.csv")